# **OPTIMISATION OF U's and D's**

This section shows how the U'S and V's can be optimzed via the Advanced Optimization Algorithms

In [ ]:
import numpy as np
import random

 # **1. Simulated Anealing Method**

In [ ]:
def simulate_and_optimize(N, true_u, true_d, X0=1.0, initial_u=1.0, initial_d=1.0, temp=50, cooling_rate=0.99, max_iterations=1000, random_seed=88):
    # Set random seed for reproducibility
    np.random.seed(random_seed)
    random.seed(random_seed)

    # Parameters for the simulation
    T = N  # Total time
    dt = T / N  # Time step size

    # Initialize the process
    X = np.zeros(N+1)
    X[0] = X0

    # Generate standard normal random variables
    Z = np.random.normal(0, 1, N)

    # Simulate the process using Euler-Maruyama with true parameters
    for j in range(N):
        X[j+1] = X[j] + 0.5 * np.log(true_u * true_d) * X[j] * dt + (1/(2*dt)) * np.log(true_u/true_d) * X[j] * (Z[j] / np.sqrt(N))

    # Compute the increments
    Delta_X = np.diff(X)

    # Define the objective function
    def objective_function(u, d):
        J = 0
        for j in range(N):
            predicted_increment = 0.5 * np.log(u * d) * X[j] * dt + (1/(2*dt)) * np.log(u/d) * X[j] * (Z[j] / np.sqrt(N))
            J += (Delta_X[j] - predicted_increment)**2
        return J

    # Simulated Annealing algorithm
    def simulated_annealing(objective_function, initial_u, initial_d, temp, cooling_rate, max_iterations):
        current_u = initial_u
        current_d = initial_d
        current_objective = objective_function(current_u, current_d)
        best_u, best_d = current_u, current_d
        best_objective = current_objective

        for i in range(max_iterations):
            # Generate new candidate solutions by adding small perturbations
            candidate_u = current_u + np.random.normal(0, 0.1)
            candidate_d = current_d + np.random.normal(0, 0.1)

            # Ensure candidate_u and candidate_d remain positive
            candidate_u = max(candidate_u, 1e-6)
            candidate_d = max(candidate_d, 1e-6)

            candidate_objective = objective_function(candidate_u, candidate_d)

            # Calculate acceptance probability
            acceptance_prob = np.exp((current_objective - candidate_objective) / temp)

            # Accept the new solution with a certain probability
            if candidate_objective < current_objective or random.random() < acceptance_prob:
                current_u, current_d = candidate_u, candidate_d
                current_objective = candidate_objective

                # Update the best solution found
                if candidate_objective < best_objective:
                    best_u, best_d = candidate_u, candidate_d
                    best_objective = candidate_objective

            # Cool down the temperature
            temp *= cooling_rate

        return best_u, best_d, best_objective

    # Run the simulated annealing algorithm
    best_u, best_d, best_objective = simulated_annealing(objective_function, initial_u, initial_d, temp, cooling_rate, max_iterations)

    return best_u, best_d, best_objective

In [ ]:
# Example usage
true_u = 1.15
true_d = 0.95
N_values = [21, 50, 100, 150]

for N in N_values:
    best_u, best_d, best_objective = simulate_and_optimize(N, true_u, true_d)

    mse_u = (best_u - true_u)**2
    mse_d = (best_d - true_d)**2

    print(f"N={N}")
    print("Estimated u:", best_u)
    print("Estimated d:", best_d)
    print("MSE for u:", mse_u)
    print("MSE for d:", mse_d)
    print("Best objective function value:", best_objective)
    print("\n")

N=21
Estimated u: 1.180776361119313
Estimated d: 0.9158832342522818
MSE for u: 0.0009471844037463631
MSE for d: 0.001163953705084675
Best objective function value: 0.004137689458090453


N=50
Estimated u: 1.1621341510838927
Estimated d: 0.940404099514906
MSE for u: 0.00014723762252673729
MSE for d: 9.208130611982642e-05
Best objective function value: 0.0015102877830256178


N=100
Estimated u: 1.1434992419228456
Estimated d: 0.9580642920370058
MSE for u: 4.2259855577687314e-05
MSE for d: 6.50328060581163e-05
Best objective function value: 0.09657868597732967


N=150
Estimated u: 1.1505370659347014
Estimated d: 0.9495170503149012
MSE for u: 2.8843981821678097e-07
MSE for d: 2.3324039833699704e-07
Best objective function value: 0.010034070035369266




# **2. Grey Wolf Optimization**

In [ ]:
# Define the Gray Wolf Optimization algorithm
def gray_wolf_optimization(objective_function, n_wolves, max_iterations, lower_bound, upper_bound):
    # Initialize the positions of the wolves
    dim = 2  # We have two parameters to optimize: u and d
    alpha_pos = np.zeros(dim)
    beta_pos = np.zeros(dim)
    delta_pos = np.zeros(dim)
    alpha_score = float('inf')
    beta_score = float('inf')
    delta_score = float('inf')

    wolves_pos = lower_bound + np.random.rand(n_wolves, dim) * (upper_bound - lower_bound)

    # Main loop
    for t in range(max_iterations):
        for i in range(n_wolves):
            fitness = objective_function(wolves_pos[i, 0], wolves_pos[i, 1])

            if fitness < alpha_score:
                alpha_score = fitness
                alpha_pos = wolves_pos[i].copy()

            if alpha_score < fitness < beta_score:
                beta_score = fitness
                beta_pos = wolves_pos[i].copy()

            if beta_score < fitness < delta_score:
                delta_score = fitness
                delta_pos = wolves_pos[i].copy()

        a = 2 - t * (2 / max_iterations)

        for i in range(n_wolves):
            for j in range(dim):
                r1 = random.random()
                r2 = random.random()
                A1 = 2 * a * r1 - a
                C1 = 2 * r2
                D_alpha = abs(C1 * alpha_pos[j] - wolves_pos[i, j])
                X1 = alpha_pos[j] - A1 * D_alpha

                r1 = random.random()
                r2 = random.random()
                A2 = 2 * a * r1 - a
                C2 = 2 * r2
                D_beta = abs(C2 * beta_pos[j] - wolves_pos[i, j])
                X2 = beta_pos[j] - A2 * D_beta

                r1 = random.random()
                r2 = random.random()
                A3 = 2 * a * r1 - a
                C3 = 2 * r2
                D_delta = abs(C3 * delta_pos[j] - wolves_pos[i, j])
                X3 = delta_pos[j] - A3 * D_delta

                wolves_pos[i, j] = (X1 + X2 + X3) / 3

    best_wolf_pos = alpha_pos
    best_wolf_score = alpha_score

    return best_wolf_pos, best_wolf_score

# Simulate and optimize using Gray Wolf Optimization
def simulate_and_optimize_gwo(N, true_u, true_d, X0=1.0, n_wolves=5, max_iterations=1000, lower_bound=1e-6, upper_bound=2.0, random_seed=42):
    # Set random seed for reproducibility
    np.random.seed(random_seed)
    random.seed(random_seed)

    # Parameters for the simulation
    T = N  # Total time
    dt = T / N  # Time step size

    # Initialize the process
    X = np.zeros(N+1)
    X[0] = X0

    # Generate standard normal random variables
    Z = np.random.normal(0, 1, N)

    # Simulate the process using Euler-Maruyama with true parameters
    for j in range(N):
        X[j+1] = X[j] + 0.5 * np.log(true_u * true_d) * X[j] * dt + (1/(2*dt)) * np.log(true_u/true_d) * X[j] * (Z[j] / np.sqrt(N))

    # Compute the increments
    Delta_X = np.diff(X)

    # Define the objective function
    def objective_function(u, d):
        J = 0
        for j in range(N):
            predicted_increment = 0.5 * np.log(u * d) * X[j] * dt + (1/(2*dt)) * np.log(u/d) * X[j] * (Z[j] / np.sqrt(N))
            J += (Delta_X[j] - predicted_increment)**2
        return J

    # Run the Gray Wolf Optimization algorithm
    best_params, best_objective = gray_wolf_optimization(objective_function, n_wolves, max_iterations, lower_bound, upper_bound)

    best_u, best_d = best_params
    return best_u, best_d, best_objective

In [ ]:
true_u = 1.15
true_d = 0.95
N_values = [21, 50, 100, 150]

for N in N_values:
    best_u, best_d, best_objective = simulate_and_optimize_gwo(N, true_u, true_d, n_wolves=5, max_iterations=1000)

    mse_u = (best_u - true_u)**2
    mse_d = (best_d - true_d)**2

    print(f"N={N}")
    print("Estimated u:", best_u)
    print("Estimated d:", best_d)
    print("MSE for u:", mse_u)
    print("MSE for d:", mse_d)
    print("Best objective function value:", best_objective)
    print("\n")

<ipython-input-13-308730a80f41>:91: RuntimeWarning: invalid value encountered in log
  predicted_increment = 0.5 * np.log(u * d) * X[j] * dt + (1/(2*dt)) * np.log(u/d) * X[j] * (Z[j] / np.sqrt(N))


N=21
Estimated u: 1.149592874980421
Estimated d: 0.9502348960111736
MSE for u: 1.6575078156716674e-07
MSE for d: 5.517613606529505e-08
Best objective function value: 3.373940578368689e-07


N=50
Estimated u: 1.1505482303757104
Estimated d: 0.9495734719242667
MSE for u: 3.005565448516772e-07
MSE for d: 1.8192619938874325e-07
Best objective function value: 2.5753828692767273e-06


N=100
Estimated u: 1.150878111311479
Estimated d: 0.9492351357427985
MSE for u: 7.71079475347509e-07
MSE for d: 5.850173319443389e-07
Best objective function value: 0.00022342735226436349


N=150
Estimated u: 1.1504680794136517
Estimated d: 0.9495839431782126
MSE for u: 2.1909833748460317e-07
MSE for d: 1.7310327895579768e-07
Best objective function value: 0.005820722292058322




# **3. particle swarm optimization**

In [ ]:
# Define the Particle Swarm Optimization algorithm
def particle_swarm_optimization(objective_function, n_particles, max_iterations, lower_bound, upper_bound):
    dim = 2  # We have two parameters to optimize: u and d
    w = 0.5  # Inertia weight
    c1 = 1.5  # Cognitive parameter
    c2 = 1.5  # Social parameter

    # Initialize the particles
    particles_pos = lower_bound + np.random.rand(n_particles, dim) * (upper_bound - lower_bound)
    particles_vel = np.random.rand(n_particles, dim)
    personal_best_pos = particles_pos.copy()
    personal_best_score = np.array([objective_function(ind[0], ind[1]) for ind in particles_pos])

    global_best_idx = np.argmin(personal_best_score)
    global_best_pos = personal_best_pos[global_best_idx]
    global_best_score = personal_best_score[global_best_idx]

    for iteration in range(max_iterations):
        for i in range(n_particles):
            r1, r2 = np.random.rand(dim), np.random.rand(dim)
            particles_vel[i] = (w * particles_vel[i] +
                                c1 * r1 * (personal_best_pos[i] - particles_pos[i]) +
                                c2 * r2 * (global_best_pos - particles_pos[i]))
            particles_pos[i] += particles_vel[i]

            # Ensure positions are within the specified bounds
            particles_pos[i] = np.clip(particles_pos[i], lower_bound, upper_bound)

            # Update personal best
            score = objective_function(particles_pos[i][0], particles_pos[i][1])
            if score < personal_best_score[i]:
                personal_best_pos[i] = particles_pos[i].copy()
                personal_best_score[i] = score

                # Update global best
                if score < global_best_score:
                    global_best_pos = particles_pos[i].copy()
                    global_best_score = score

    return global_best_pos, global_best_score

# Simulate and optimize using Particle Swarm Optimization
def simulate_and_optimize_pso(N, true_u, true_d, X0=1.0, n_particles=30, max_iterations=1000, lower_bound=1e-6, upper_bound=2.0, random_seed=42):
    # Set random seed for reproducibility
    np.random.seed(random_seed)
    random.seed(random_seed)

    # Parameters for the simulation
    T = N  # Total time
    dt = T / N  # Time step size

    # Initialize the process
    X = np.zeros(N+1)
    X[0] = X0

    # Generate standard normal random variables
    Z = np.random.normal(0, 1, N)

    # Simulate the process using Euler-Maruyama with true parameters
    for j in range(N):
        X[j+1] = X[j] + 0.5 * np.log(true_u * true_d) * X[j] * dt + (1/(2*dt)) * np.log(true_u/true_d) * X[j] * (Z[j] / np.sqrt(N))

    # Compute the increments
    Delta_X = np.diff(X)

    # Define the objective function
    def objective_function(u, d):
        J = 0
        for j in range(N):
            predicted_increment = 0.5 * np.log(u * d) * X[j] * dt + (1/(2*dt)) * np.log(u/d) * X[j] * (Z[j] / np.sqrt(N))
            J += (Delta_X[j] - predicted_increment)**2
        return J

    # Run the Particle Swarm Optimization algorithm
    best_params, best_objective = particle_swarm_optimization(objective_function, n_particles, max_iterations, lower_bound, upper_bound)

    best_u, best_d = best_params
    return best_u, best_d, best_objective

In [ ]:
true_u = 1.15
true_d = 0.95
N_values = [21, 50, 100, 150]

for N in N_values:
    best_u, best_d, best_objective = simulate_and_optimize_pso(N, true_u, true_d, n_particles=30, max_iterations=1000)

    mse_u = (best_u - true_u)**2
    mse_d = (best_d - true_d)**2

    print(f"N={N}")
    print("Estimated u:", best_u)
    print("Estimated d:", best_d)
    print("MSE for u:", mse_u)
    print("MSE for d:", mse_d)
    print("Best objective function value:", best_objective)
    print("\n")

N=21
Estimated u: 1.15
Estimated d: 0.95
MSE for u: 0.0
MSE for d: 0.0
Best objective function value: 5.0016400655736515e-31


N=50
Estimated u: 1.15
Estimated d: 0.95
MSE for u: 0.0
MSE for d: 0.0
Best objective function value: 1.5223994727895003e-30


N=100
Estimated u: 1.15
Estimated d: 0.95
MSE for u: 0.0
MSE for d: 0.0
Best objective function value: 2.2600210118400897e-28


N=150
Estimated u: 1.15
Estimated d: 0.95
MSE for u: 0.0
MSE for d: 0.0
Best objective function value: 1.564535940707426e-26




# **4. differential evolution**

In [ ]:
# Define the Differential Evolution algorithm
def differential_evolution(objective_function, population_size, max_iterations, lower_bound, upper_bound, F=0.8, CR=0.9):
    dim = 2  # We have two parameters to optimize: u and d

    # Initialize the population
    population = lower_bound + np.random.rand(population_size, dim) * (upper_bound - lower_bound)
    best_individual = np.zeros(dim)
    best_score = float('inf')

    for iteration in range(max_iterations):
        for i in range(population_size):
            # Mutation
            indices = [idx for idx in range(population_size) if idx != i]
            a, b, c = population[np.random.choice(indices, 3, replace=False)]
            mutant_vector = np.clip(a + F * (b - c), lower_bound, upper_bound)

            # Crossover
            crossover_vector = np.array([mutant_vector[j] if random.random() < CR else population[i][j] for j in range(dim)])

            # Selection
            trial_score = objective_function(crossover_vector[0], crossover_vector[1])
            target_score = objective_function(population[i][0], population[i][1])
            if trial_score < target_score:
                population[i] = crossover_vector
                if trial_score < best_score:
                    best_score = trial_score
                    best_individual = crossover_vector.copy()

    return best_individual, best_score

# Simulate and optimize using Differential Evolution
def simulate_and_optimize_de(N, true_u, true_d, X0=1.0, population_size=50, max_iterations=1000, lower_bound=1e-6, upper_bound=2.0, F=0.8, CR=0.9, random_seed=82):
    # Set random seed for reproducibility
    np.random.seed(random_seed)
    random.seed(random_seed)

    # Parameters for the simulation
    T = N  # Total time
    dt = T / N  # Time step size

    # Initialize the process
    X = np.zeros(N+1)
    X[0] = X0

    # Generate standard normal random variables
    Z = np.random.normal(0, 1, N)

    # Simulate the process using Euler-Maruyama with true parameters
    for j in range(N):
        X[j+1] = X[j] + 0.5 * np.log(true_u * true_d) * X[j] * dt + (1/(2*dt)) * np.log(true_u/true_d) * X[j] * (Z[j] / np.sqrt(N))

    # Compute the increments
    Delta_X = np.diff(X)

    # Define the objective function
    def objective_function(u, d):
        J = 0
        for j in range(N):
            predicted_increment = 0.5 * np.log(u * d) * X[j] * dt + (1/(2*dt)) * np.log(u/d) * X[j] * (Z[j] / np.sqrt(N))
            J += (Delta_X[j] - predicted_increment)**2
        return J

    # Run the Differential Evolution algorithm
    best_params, best_objective = differential_evolution(objective_function, population_size, max_iterations, lower_bound, upper_bound, F, CR)

    best_u, best_d = best_params
    return best_u, best_d, best_objective

# Example usage
N = 20
true_u = 1.15
true_d = 0.95

best_u, best_d, best_objective = simulate_and_optimize_de(N, true_u, true_d, population_size=50, max_iterations=1000)


In [ ]:
true_u = 1.15
true_d = 0.95
N_values = [21, 50, 100, 150]

for N in N_values:
    best_u, best_d, best_objective = simulate_and_optimize_de(N, true_u, true_d, population_size=50, max_iterations=1000)

    mse_u = (best_u - true_u)**2
    mse_d = (best_d - true_d)**2

    print(f"N={N}")
    print("Estimated u:", best_u)
    print("Estimated d:", best_d)
    print("MSE for u:", mse_u)
    print("MSE for d:", mse_d)
    print("Best objective function value:", best_objective)
    print("\n")

N=21
Estimated u: 1.15
Estimated d: 0.95
MSE for u: 0.0
MSE for d: 0.0
Best objective function value: 2.9909692036333773e-31


N=50
Estimated u: 1.1500000000000001
Estimated d: 0.9499999999999997
MSE for u: 4.930380657631324e-32
MSE for d: 4.930380657631324e-32
Best objective function value: 6.039042230117836e-30


N=100
Estimated u: 1.15
Estimated d: 0.95
MSE for u: 0.0
MSE for d: 0.0
Best objective function value: 4.717537954274827e-28


N=150
Estimated u: 1.1499999999999997
Estimated d: 0.9500000000000002
MSE for u: 4.930380657631324e-32
MSE for d: 4.930380657631324e-32
Best objective function value: 2.1343320695895582e-26


